In [1]:
!pip install pandas datasets unsloth transformers accelerate bitsandbytes -q


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch-tensorrt 2.5.0a0 requires torch<2.6.0,>=2.5.0.dev, but you have torch 2.7.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch
from tqdm import tqdm


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: If you want to finetune Gemma 2, upgrade flash-attn to version 2.6.3 or higher!
Newer versions support faster and less memory usage kernels for Gemma 2's attention softcapping!
To update flash-attn, do the below:

pip install --no-deps --upgrade "flash-attn>=2.6.3"
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
# Load your CSV (make sure the file 'medquad_tamil.csv' is uploaded)
df = pd.read_csv("medquad_tamil.csv").dropna()

# Create a formatted 'text' column for the LLM
df["text"] = "### கேள்வி:\n" + df["question_tamil"] + "\n\n### பதில்:\n" + df["answer_tamil"]

# Split into train and test datasets (keeping original columns)
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert to HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))


In [4]:
# Load the model from Hugging Face
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="abhinand/gemma-2b-it-tamil-v0.1-alpha",  # HuggingFace repo ID
    max_seq_length=512,
    dtype=torch.float16,
    load_in_4bit=True,
    device_map="auto",
)

# Fix padding token
tokenizer.pad_token = tokenizer.eos_token
model.eval()


==((====))==  Unsloth 2025.6.3: Fast Gemma patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████████████| 2/2 [00:02<00:00,  1.36s/it]


GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaFixedRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): G

In [7]:
from tqdm import tqdm

generated_answers = []
reference_answers = []

subset = test_dataset.select(range(100))

for item in tqdm(subset):
    question = item["question_tamil"]
    reference = item["answer_tamil"]

    prompt = f"""கேள்வி:
{question}

பதில்:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Extract answer
    if "பதில்:" in decoded:
        answer = decoded.split("பதில்:")[-1].strip()
    else:
        answer = decoded.replace(prompt, "").strip()

    # Fallback for empty answers
    if not answer:
        print(f"⚠️ No answer for question: {question}")
        print("🔍 Full decoded output:\n", decoded)
        answer = "மன்னிக்கவும், இந்த கேள்விக்கு பதில் இல்லை."  # Default fallback
    else:
        print(f"✅ Answer generated for: {question}")

    generated_answers.append(answer)
    reference_answers.append(reference)


  1%|▍                                          | 1/100 [00:08<13:51,  8.40s/it]

✅ Answer generated for: இளைஞர் ஹண்டிங்டன் நோய்த்தொற்று என்ன?


  2%|▊                                          | 2/100 [00:19<15:56,  9.76s/it]

✅ Answer generated for: பெர்ரி நோய்க்கான சிகிச்சைகள் என்ன?


  3%|█▎                                         | 3/100 [00:21<10:14,  6.34s/it]

...

✅ Answer generated for: க்ளவுகோமாவுக்கு என்ன சிகிச்சைகள்?


 99%|█████████████████████████████████████████▌| 99/100 [11:56<00:05,  5.94s/it]

✅ Answer generated for: ஓஸ்டியோபிரோஸ் 7 - ன் அறிகுறிகள் என்ன?


100%|█████████████████████████████████████████| 100/100 [12:03<00:00,  7.24s/it]

✅ Answer generated for: பல மையமான ஓட்டோலிசிஸ், நெடுஞ்சாலை, தொற்று நோய் போன்ற மரபணு மாற்றங்கள் என்னவாகும்?


In [9]:
!pip install bert_score

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [10]:
from bert_score import score

P, R, F1 = score(generated_answers, reference_answers, lang="ta")  # ta = Tamil

print(f"Precision: {P.mean().item():.4f}")
print(f"Recall:    {R.mean().item():.4f}")
print(f"F1 Score:  {F1.mean().item():.4f}")


Precision: 0.5992
Recall:    0.6276
F1 Score:  0.6110


In [11]:
import pandas as pd

# Convert tensors to numpy
P = P.numpy()
R = R.numpy()
F1 = F1.numpy()

# Report average
print(f"\n✅ BERTScore Precision: {P.mean():.4f}")
print(f"✅ BERTScore Recall:    {R.mean():.4f}")
print(f"✅ BERTScore F1:        {F1.mean():.4f}")

# Optional: Save individual results
results_df = pd.DataFrame({
    "Generated Answer": generated_answers,
    "Reference Answer": reference_answers,
    "Precision": P,
    "Recall": R,
    "F1 Score": F1
})

results_df.to_csv("not_tamil_bertscore.csv", index=False)
print("✅ Saved: not_tamil_bertscore.csv")



✅ BERTScore Precision: 0.5992
✅ BERTScore Recall:    0.6276
✅ BERTScore F1:        0.6110
✅ Saved: not_tamil_bertscore.csv
